# Intuitive Algorithm Solver

### Configuration

In [1]:
FILEPATH = r'./../brightspace-locker/Data assignment parcel transport 2 Small.xlsx'
# FILEPATH = r'./../other/Data assignment parcel transport 2 very small Small.xlsx'

### Initialisations

In [2]:
from graph_visualiser import visualise_graph
from integer_linear_problem import Ilp
from typing import Dict, List, Set
from tqdm import tqdm
import pandas as pd
import numpy as np
import copy
import json
from config import Config

ILP = Ilp.from_excel(FILEPATH)
CONFIG = Config()

### Solver

In [3]:
def powerset(s: Set) -> Set:
    bitmasks = [1 << i for i in range(len(s))] # 1 << i is basically equal to 2^i
    # the n-th digits in the binary representation of bit_selection dictates whether the n-th item is selected (1 = selected, 0 = not selected)
    for bit_selection in range(1, 1 << len(s)): # start with 1 to prevent empty set
        yield {el for el, bitmask in zip(s, bitmasks) if bit_selection & bitmask}

def REMOVE():
    z_min = float('inf')

    x = False # TODO: REMOVE
    fixHubLength = False # TODO: REMOVE
    with tqdm(total=(1 << len(ILP.N)) - 1, ncols=CONFIG.TQDM_NCOLS) as pbar:
        for hubs in powerset(ILP.N):
            pbar.update(1)

            non_hubs = ILP.N - hubs

            H = dict()
            for node in ILP.N:
                H[node] = 1 if node in hubs else 0

            E = {node: dict() for node in ILP.N}
            
            # non-hub to hub / hub to non-hub edges
            for non_hub in non_hubs:
                cost_dict = {hub: ILP.c[non_hub][hub] for hub in hubs} # filter on hubs
                min_cost_hub = min(cost_dict, key=cost_dict.get)
                for hub in hubs:
                    E[non_hub][hub] = 1 if hub == min_cost_hub else 0
                    E[hub][non_hub] = 1 if hub == min_cost_hub else 0 # since E_ij = E_ji for all i =/= j

            # non-hub to non-hub edges (including itself)
            for non_hub1 in non_hubs:
                for non_hub2 in non_hubs:
                    E[non_hub1][non_hub2] = 0

            # hub to hub edges (including itself)
            for hub1 in hubs:
                for hub2 in hubs:
                    E[hub1][hub2] = 1

            if fixHubLength:
                if x:
                    continue
                if len(hubs) == 6:# TODO: REMOVE
                    x = True
            else:
                current_z = ILP.get_z_multiple_hubs(H, E)
                if current_z >= z_min:
                    continue

                z_min = current_z
                # TODO: remove deepcopy and remove copy import
                hubs_min = copy.deepcopy(hubs)
                non_hubs_min = copy.deepcopy(non_hubs)
                H_min = copy.deepcopy(H)
                E_min = copy.deepcopy(E)

    print('(Possibly Sub-optimal) Solution:')
    print(f'z = {z_min}')
    print(f'Hubs = {hubs_min}')
    print(f'Non-hubs = {non_hubs_min}')
    print(f'H = {json.dumps(H_min, indent=4)}')
    print(f'E = {json.dumps(E_min, indent=4)}')
    print(ILP)
    # visualise_graph(ILP.N, H_min, E_min, ILP.c, ILP.filepath)

import cProfile
import pstats

with cProfile.Profile() as pr:
    REMOVE()

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
stats.print_stats()

100%|███████████████████████████████████████████████████████████| 1023/1023 [00:45<00:00, 22.64it/s]


(Possibly Sub-optimal) Solution:
z = 234953
Hubs = {3, 4, 7}
Non-hubs = {1, 2, 5, 6, 8, 9, 10}
H = {
    "1": 0,
    "2": 0,
    "3": 1,
    "4": 1,
    "5": 0,
    "6": 0,
    "7": 1,
    "8": 0,
    "9": 0,
    "10": 0
}
E = {
    "1": {
        "3": 1,
        "4": 0,
        "7": 0,
        "1": 0,
        "2": 0,
        "5": 0,
        "6": 0,
        "8": 0,
        "9": 0,
        "10": 0
    },
    "2": {
        "3": 0,
        "4": 1,
        "7": 0,
        "1": 0,
        "2": 0,
        "5": 0,
        "6": 0,
        "8": 0,
        "9": 0,
        "10": 0
    },
    "3": {
        "1": 1,
        "2": 0,
        "5": 1,
        "6": 0,
        "8": 0,
        "9": 0,
        "10": 0,
        "3": 1,
        "4": 1,
        "7": 1
    },
    "4": {
        "1": 0,
        "2": 1,
        "5": 0,
        "6": 1,
        "8": 0,
        "9": 0,
        "10": 0,
        "3": 1,
        "4": 1,
        "7": 1
    },
    "5": {
        "3": 1,
        "4": 0,
        "7": 0,
